In [1]:
import nltk
from collections import Counter
import pandas as pd
import numpy as np
import pickle
from nltk.corpus import stopwords
import pymorphy2
import string
from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn import svm
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from string import digits 
import math
from nltk import word_tokenize
from collections import defaultdict
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import gc
from scipy import sparse
pd.set_option('display.max_columns', 100)


from sklearn.metrics import confusion_matrix

from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
pd.set_option('display.max_rows', 100)


import os


from sklearn.datasets import make_classification
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\NurimanovA\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

# Data Collection

In [2]:
df_train = pd.read_csv('data_train.csv', encoding="utf8")

In [3]:
df_train = df_train[['subject', 'description_text', 'agent_name', 'type', 'group_name', 'Result']]
df_train['Text'] = df_train['subject'] + df_train['description_text'] 

df_train.dropna(subset=['Text'], inplace=True)
df_train.Result.value_counts()

0    3997
1     106
Name: Result, dtype: int64

In [4]:
df_train.Result.value_counts()

0    3997
1     106
Name: Result, dtype: int64

In [5]:
banned = ['Request', 'for']
f = lambda x: ' '.join([item for item in x.split() if item not in banned])
df_train["Text"] = df_train["Text"].apply(f)

# Data Preparation

In [6]:
# 1 вариант объединяю имя и дескрипшн

def remove(text):
    return text[:500]

df_train['Text'] = df_train['Text'].apply(remove)

df1 = df_train[['Text','Result']]
df1.columns = ['Description','Result']

def r_dup(sent):
    words = sent.split() 
    res = [] 
    for word in words: 
        # If condition is used to store unique string  
        # in another list 'k'  
        if (sent.count(word)>1 and (word not in res)or sent.count(word)==1): 
            res.append(word) 
    return ' '.join(res)


df1['Description'] = df1['Description'].apply(r_dup)

# df_1_o = df1.loc[df1.Result == 1].reset_index(drop=True)
# df_0_o = df1.loc[df1.Result == 0].reset_index(drop=True)
# Preprocessing

#Remove words | bag of words

# rubbish = ['Услуга','Услуги','Работа','Работы']

def text_process(mess):
    
    mess = mess.replace('/', ' ')
    mess = mess.replace('(',' ')
    mess = mess.replace(')',' ')
    mess = mess.replace('-',' ')
    
    nopunc = [char for char in mess if char not in string.punctuation]
    nopunc = ''.join(nopunc)
    
    nopunc = [word for word in nopunc.split() if word.lower() not in stopwords.words('russian')]
    return ' '.join(nopunc)

# убрать стопвордс
# убрать цифры

morph = pymorphy2.MorphAnalyzer()


def lemmatize(text):
    res = list()
    words = text.split() # разбиваем текст на слова
    for word in words:
        p = morph.parse(word)[0]
        res.append(p.normal_form)
        
    #removing numbers    
    res = [''.join(x for x in i if x.isalpha()) for i in res] 
    while '' in res:
        
        res.remove('')     
    
    return ' '.join(res)

def pos_tag(text):
    words = text.split()
    res = list()
    for word in words:
        p = morph.parse(word)[0]
        res.append(p.tag.POS)
    return res       

#remove duplicate



# df1['Lemma'] = df1['Description'].apply(r_dup)

df1['Description'] = df1['Description'].apply(r_dup)
df1['Lemma'] = df1['Description'].apply(text_process)
df1['Lemma'] = df1['Lemma'].apply(lemmatize)

print('LR l2 norm, balanced')                                
    
# df1['Description'] = df1['Description'].apply(text_process)                                 
df1['Description'] = df1['Description'].apply(lemmatize)
df1['pos_teg'] = df1['Result']
df1['pos_teg']= df1['Description'].apply(pos_tag)                                        
  

#divide reslts

df1 = df1.reset_index(drop=True)
df_1 = df1.loc[df1.Result == 1].reset_index(drop=True)
df_0 = df1.loc[df1.Result == 0].reset_index(drop=True)


C:\Users\NURIMA~1\AppData\Local\Temp/ipykernel_15744/1481684415.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['Description'] = df1['Description'].apply(r_dup)
C:\Users\NURIMA~1\AppData\Local\Temp/ipykernel_15744/1481684415.py:80: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['Description'] = df1['Description'].apply(r_dup)
C:\Users\NURIMA~1\AppData\Local\Temp/ipykernel_15744/1481684415.py:81: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using

LR l2 norm, balanced


C:\Users\NURIMA~1\AppData\Local\Temp/ipykernel_15744/1481684415.py:87: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['Description'] = df1['Description'].apply(lemmatize)
C:\Users\NURIMA~1\AppData\Local\Temp/ipykernel_15744/1481684415.py:88: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['pos_teg'] = df1['Result']
C:\Users\NURIMA~1\AppData\Local\Temp/ipykernel_15744/1481684415.py:89: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,

# Data Modeling

In [8]:
########## LOGISTIC REGRESIION ##############

X_train, X_test, Y_train, Y_test = train_test_split(df1['Description'],df1['Result'],test_size=0.15, random_state=np.random)

pipeline = Pipeline([
    ('tfidf',TfidfVectorizer(max_features=1500,
                             norm='l2', analyzer='char', stop_words="russian", ngram_range=(2,5),dtype=np.float32)),
    ('classifier', LogisticRegression(penalty='l2', class_weight='balanced'))
])

Y_train=Y_train.astype('int')
Y_test=Y_test.astype('int')
df1['Result'] = df1['Result'].astype('int')

#pipeline.fit(df1['Description'], df1['Result'])
pipeline.fit(X_train, Y_train)

Y_train_pred = (pipeline.predict_proba(X_train)[:,1]>=0.4).astype(float)
Y_test_pred = (pipeline.predict_proba(X_test)[:,1]>=0.4).astype(float)

cmtr = confusion_matrix(Y_train, Y_train_pred) 
print("Confusion Matrix Training:\n", cmtr) 

acctr = accuracy_score(Y_train, Y_train_pred) 
print("Accurracy Training:", acctr) 

rcl = recall_score(Y_train, Y_train_pred)
print("Recall Train:", rcl) 

cmte = confusion_matrix(Y_test, Y_test_pred) 
print("Confusion Matrix Testing:\n", cmte) 

accte = accuracy_score(Y_test, Y_test_pred) 
print("Accurracy Test:", accte) 

rcl = recall_score(Y_test, Y_test_pred)
print("Recall Test:", rcl) 

Confusion Matrix Training:
 [[2918  483]
 [   3   83]]
Accurracy Training: 0.8606251792371666
Recall Train: 0.9651162790697675
Confusion Matrix Testing:
 [[518  78]
 [  4  16]]
Accurracy Test: 0.8668831168831169
Recall Test: 0.8


In [9]:
########## LOGISTIC REGRESIION ##############

#X_train, X_test, Y_train, Y_test = train_test_split(df1['Description'],df1['Result'],test_size=0.2, random_state=np.random)

pipeline = Pipeline([
    ('tfidf',TfidfVectorizer(max_features=1500,
                             norm='l2', analyzer='char', stop_words="russian", ngram_range=(2,5),dtype=np.float32)),
    ('classifier', LogisticRegression(penalty='l2', class_weight='balanced'))
])

Y_train=Y_train.astype('int')
Y_test=Y_test.astype('int')
df1['Result'] = df1['Result'].astype('int')

#pipeline.fit(df1['Description'], df1['Result'])
pipeline.fit(df1['Description'],df1['Result']) 

Pipeline(steps=[('tfidf',
                 TfidfVectorizer(analyzer='char', dtype=<class 'numpy.float32'>,
                                 max_features=1500, ngram_range=(2, 5),
                                 stop_words='russian')),
                ('classifier', LogisticRegression(class_weight='balanced'))])